In [1]:
import pandas as pd

import BiblioMeter_Utils as bmu

La cellule ci dessous récupère le nom de chaque feuille du document excel fourni par Jenny et créer une liste nommée list_annee les contenant dans des sous listes de chaque année

In [2]:
Mois = ['01','02','03','04','05','06','07','08','09','10','11','12']

Annees = ['2019','2020','2021']

annee_2019 = []
annee_2020 = []
annee_2021 = []

fichier_RH = pd.ExcelFile(bmu.PATH_TO_EFFECTIFS)
sheet_names=fichier_RH.sheet_names
sheet_names.reverse()


for i in sheet_names:
    if '2019' in i:
        annee_2019.append(i)
    if '2020' in i:
        annee_2020.append(i)
    if '2021' in i:
        annee_2021.append(i)

list_annee = [annee_2019,annee_2020,annee_2021]

La cellule ci dessous va lire le fichier Effectifs.xlsx pour l'exporter sour un format DF exploitable avec pandas sur python

list_de_list_de_df : est une liste de liste de taille mois*année qui contient les informations des employers du LITEN sous forme de DF

Lors de cette extraction nous rajoutons une colonne de la forme 'mmaaaa' pour savoir de quelle année vient l'information personnel

In [3]:
list_de_list_de_df = []
for i in range(len(list_annee)):
    list_de_df = []
    for j in list_annee[i]:
        df = pd.read_excel(bmu.PATH_TO_EFFECTIFS, sheet_name=j)
        df['mmaaaa'] = [j] * df.shape[0]
        list_de_df.append(df)
    
    list_de_list_de_df.append(list_de_df)

La cellule ci dessous reprend list_de_list_de_df, pour créer une liste longue du nombre d'année et qui rassemble toutes les pages du fichier Effectfs.xlsx de chaque mois par année

In [4]:
list_de_df_par_annee = []

for i in range(len(list_annee)):
    df_par_annee = pd.DataFrame()
    for j in range(len(list_annee[i])):
        df_par_annee = df_par_annee.append(list_de_list_de_df[i][j])
        df_par_annee.reset_index(drop=True, inplace=True)
    list_de_df_par_annee.append(df_par_annee)

Celle ci rassemble tout le document sur la même DF avec duplication du personnel

df_effectifs : df de tous les documents RH

In [5]:
df_effectifs = pd.DataFrame()

for i in range(len(list_de_df_par_annee)):
    df_effectifs = df_effectifs.append(list_de_df_par_annee[i])

La cellule de dessous reprendre list_de_df_par_annee et rassemble les matricules identiques sur la même ligne et concatenate en une seule liste 'mmaaaa' et 'Dpt/DOB (lib court)'

In [6]:
df = df_effectifs

dg_mmaaa = pd.DataFrame.from_dict({x[0]:[x[1]['mmaaaa'].tolist()] for x in df.groupby(['Matricule'])}).T
dg_mmaaa.index.name = 'Matricule'
dg_mmaaa.columns = ['list of mmaaaa']
dg_mmaaa.reset_index(inplace = True)

dg_depart = pd.DataFrame.from_dict({x[0]:[x[1]['Dpt/DOB (lib court)'].tolist()] for x in df.groupby(['Matricule'])}).T
dg_depart.index.name = 'Matricule'
dg_depart.columns = ['list of Dpt/DOB (lib court)']
dg_depart.reset_index(inplace = True)

df = pd.merge(df, dg_mmaaa, how = 'left', left_on = ['Matricule'], right_on = ['Matricule'])
df = pd.merge(df, dg_depart, how = 'left', left_on = ['Matricule'], right_on = ['Matricule'])

In [9]:
# Récupérer les auteurs LITEN
df_authorsinst = pd.read_csv(bmu.PATH_DAT_DEDUPLICATED + 'authorsinst.dat', 
                 sep="\t")

df_authors = pd.read_csv(bmu.PATH_DAT_DEDUPLICATED + 'authors.dat', 
                 sep="\t")

# Et les associés aux publications
df_articles = pd.read_csv(bmu.PATH_DAT_DEDUPLICATED + 'articles.dat', 
                 sep="\t")

df_authorsinst_authors = pd.merge(df_authorsinst, 
                     df_authors, 
                     how = 'left', left_on = ['Pub_id','Idx_author'], right_on = ['Pub_id','Idx_author'])

filt_authors_LITEN = (df_authorsinst_authors['Secondary_institutions'] == 'LITEN')
list_authors_LITEN = set(df_authorsinst_authors[filt_authors_LITEN]['Co_author'].tolist())

merged_df_Liten = pd.merge(df_authorsinst_authors[filt_authors_LITEN], 
                     df_articles, 
                     how = 'left', left_on = ['Pub_id'], right_on = ['Pub_id'])

merged_df_Liten['Co_author'] = merged_df_Liten['Co_author'].str.upper()
merged_df_Liten['Co_author'] = merged_df_Liten['Co_author'].str.split()

# Ci dessous sert à créer une colonne et à mettre sous le bon format le nom des co auteurs.

merged_df_Liten['Co_author_joined'] = merged_df_Liten['Co_author']
for i in range(len(merged_df_Liten)):
    length = len(merged_df_Liten['Co_author'][i])
    merged_df_Liten['Co_author_joined'].iloc[i] = (' ').join(merged_df_Liten['Co_author'][i][0:length-1])
    
print('Done')

C:\Users\ld259969\Documents\PyVenv\BiblioMeterDraft\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
# Création du fichier RH epuré
fichier_rh = df.drop_duplicates(subset='Matricule')

In [11]:
# Selection des colonness pratique pour une visu plus lisible
# CES COLONNES NE SONT PAS LES COLONNES DEFINITIVES, CEST SIMPLEMENT POUR RENDRE LISIBLE LE FICHIER EXCEL DE TEST

ce_que_je_veux_voir_publi = ['Pub_id', 'Idx_author', 'Authors', 'Co_author_joined', 'DOI', 'ISSN', 'Secondary_institutions']
df_Liten = merged_df_Liten[ce_que_je_veux_voir_publi]

ce_que_je_veux_voir_RH =['Matricule', 'Nom', 'Prénom', 'Dpt/DOB (lib court)',
       'Service (lib court)', 'Laboratoire (lib court)', 'Laboratoire (lib long)']
df_fichierRH = fichier_rh[ce_que_je_veux_voir_RH]

#### Ici on crée dj_jp, df destinée à être envoyée vers un fichier excel pour JP après quelques traitements

In [12]:
# Required imports for similarity function
from difflib import SequenceMatcher
import numpy as np
import itertools
similarity = lambda x: np.mean([SequenceMatcher(None, a,b).ratio() for a,b in itertools.combinations(x, 2)])

df_jp = pd.DataFrame() # La df qui retrouve les personnes dans le fichier RH
df_orphelin = pd.DataFrame() # La df qui retrouve pas les personnes dans le fichier RH
counter = [] # Une liste pour plus tard qui nous permettra de colorer les bonnes lignes

for i in range(len(merged_df_Liten)):

    df_inter_merged = merged_df_Liten.iloc[i]
    df_inter_rh = fichier_rh[fichier_rh['Nom'] == df_inter_merged['Co_author_joined']]

    if len(df_inter_rh) != 0:

        list_similarity = []

        _ = [list_similarity.append(similarity([df_inter_rh['Prénom'].iloc[j], df_inter_merged['Co_author'][1]])) for j in range(len(df_inter_rh))]

        emplacement_du_max = [i for i, x in enumerate(list_similarity) if x == max(list_similarity)]

        if max(list_similarity) == 0:
            df_orphelin = df_orphelin.append(df_inter_merged)

        if len(list_similarity) > 1 and len(emplacement_du_max) > 1:

            for j in emplacement_du_max:
                df_inter_jp = pd.DataFrame()
                df_inter_jp = pd.merge(df_inter_merged.to_frame().T, 
                                 df_inter_rh.iloc[j].to_frame().T, 
                                 how = 'left', left_on = ['Co_author_joined'], right_on = ['Nom'])
                df_jp = df_jp.append(df_inter_jp, ignore_index = True)
                counter.append(len(df_jp))
        else:
            df_inter_jp = pd.DataFrame()
            df_inter_jp = pd.merge(df_inter_merged.to_frame().T, 
                                 df_inter_rh.iloc[emplacement_du_max], 
                                 how = 'left', left_on = ['Co_author_joined'], right_on = ['Nom'])
            df_jp = df_jp.append(df_inter_jp, ignore_index = True)

    else:
        df_orphelin = df_orphelin.append(df_inter_merged)

print('Done')

Done


#### LA ON VA ESSAYER D'AJOUTER LES OTPs (nom de la fonction à changer)

In [13]:
_=[bmu.you_got_OTPed(df_jp,i) for i in range(len(df_jp))]

#### Ca fonctionne, mais pas encore avec le choix multiple, on va le rajouter par dessus ! (Dans le workbook)

In [14]:
ce_que_je_veux_voir_JP = ce_que_je_veux_voir_publi + ce_que_je_veux_voir_RH + ['List_of_OTP']
ce_que_je_veux_voir_JP.remove('Pub_id')
df_jp.set_index(['Pub_id'], inplace = True)

In [15]:
# Creating the workbook
from openpyxl import Workbook, load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill

wb = Workbook()
ws = wb.active
ws.title = 'Publi x Effectifs'

ft = PatternFill(fgColor = '00FFFF00', fill_type = "solid")

for r in dataframe_to_rows(df_jp[ce_que_je_veux_voir_JP], index=True, header=True):
    ws.append(r)
    last_row = ws[ws.max_row]
    if ws.max_row-2 in counter:
        for col in range(len(last_row)):
            cell = last_row[col]
            cell.fill = ft

for cell in ws['A'] + ws[1]:
    cell.style = 'Pandas'

for r in range(0,ws.max_row-2):
    
    validation_list = bmu.liste_de_validation(df_jp,r)
    
    data_val = DataValidation(type="list",formula1=validation_list)
    ws.add_data_validation(data_val)

    data_val.add(ws["O"+str(r+3)])

wb.save(bmu.PATH_JP + 'PubliXEffectifs_version_6.xlsx')

print('Done')

Done


#### Insatifait du choix multiple, il n'est pas multiple... solution trouvé avec Kutools for excel, mais à explorer encore autre part

#### Le code couleur pour le moment veut dire qu'il faut faire un choix dans les lignes jaunes consécutives